In [ ]:
!pip install skforecast

In [2]:
import pandas as pd
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
import shutil
import re
import traceback
%matplotlib inline

In [3]:
# one-hot encoding for categorical features for transformer
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        make_column_selector(dtype_include=['category']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

# Lags grid
lags_grid = [24, 48, [1, 2, 3, 23, 24, 25, 125, 127, 128]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [4]:
def search_hyperparameters(data, end_train, end_valid, one_hot_encoder, exog_features):

  # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 48,
        transformer_exog = one_hot_encoder,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 26,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )

    best_params = results_search['params'].iat[0]
    metric = results_search['metric'].iat[0]

    return best_params, metric, forecaster

def train_and_predict(best_params, end_valid, valid_num, one_hot_encoder, hrbnz01):

  forecaster = ForecasterAutoreg(
  regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
  lags               = 72,
  transformer_exog   = one_hot_encoder,
  fit_kwargs         = {"categorical_feature": "auto"}
  )
  # train the model the time series train and validation dataset
  forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
  )

  # make predictions
  predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 26
  )
  df_preds = pd.DataFrame(predictions)
  df_preds = df_preds.rename(columns={"pred": hrbnz01})

  return df_preds


In [5]:
def populate_test_data(data_dir, test_template_csv):

  # read the test template file
  df_submission = pd.read_csv(test_template_csv)
  mp_idx = df_submission.columns[1:].tolist()
  metrics = 0


  # collect all files in the directory
  filenames = os.listdir(data_dir)

  try:

    for filename in filenames:

        hrbnz01 = filename.split(".")[0].split("-")[-1]
        filepath = path.join(data_dir, filename)
        df_exog = pd.read_csv(filepath)
        df_exog["season"] = df_exog["season"].astype("category")
        df_exog["weather"] = df_exog["weather"].astype("category")
        df_exog["date"] = pd.to_datetime(df_exog["date"])
        df_exog.set_index("date", inplace=True)
        df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')

        # get the estimate end train and end validation dates
        data = df_exog.copy()
        exog_data = data.drop("gw-level", axis=1)
        exog_features = exog_data.columns
        df_idx = data.index
        train_num = int(len(data) * 0.8)
        valid_num = len(data.loc[:"2021-11-01"])
        end_train = df_idx[train_num]
        end_valid = df_idx[valid_num]

        # tune for best hyperparamters and evaluate on MAPE metric
        best_params, metric = search_hyperparameters(data, end_train, end_valid, one_hot_encoder, exog_features)
        metrics += metric

        # train and make predict into 2 years in the future of the test template
        df_predictions = train_and_predict(best_params, end_valid, valid_num, one_hot_encoder, hrbnz01)
        df_submission[hrbnz01] = df_submission.apply(
            lambda row: df_predictions.loc[row["date"]]["hrbnz01"])

        del df_exog

  except Exception as ex:
    print("[Error]")
    print(traceback.format_exc())

  mape = metrics/len(filenames)
  print(f"Mean Absolute Percentage Error: {mape}%")
  print("> Done")

  return df_submission

In [6]:
processed_data_dir = "/content/drive/MyDrive/processed_data"
test_template = "/content/drive/MyDrive/gw_test_empty.csv"

df_submission = populate_test_data(processed_data_dir, test_template)

/content/drive/MyDrive/processed_data/processed_Grundwasserstand-Monatsmittel-335547.csv
360
[Error]
Traceback (most recent call last):
  File "<ipython-input-5-6fcc7e2116b7>", line 25, in populate_test_data
    df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 6002, in __setattr__
    return object.__setattr__(self, name, value)
  File "pandas/_libs/properties.pyx", line 69, in pandas._libs.properties.AxisProperty.__set__
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/generic.py", line 730, in _set_axis
    self._mgr.set_axis(axis, labels)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/internals/managers.py", line 225, in set_axis
    self._validate_set_axis(axis, new_labels)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/internals/base.py", line 70, in _validate_set_axis
    raise ValueError(
ValueError: Length mismatch: Expec

## This is code for a single location prediction

In [24]:
df_exog = pd.read_csv("/content/processed_Grundwasserstand-Monatsmittel-326249.csv")

# set categorical columns astype category for the transformer model to auto detect them
df_exog["season"] = df_exog["season"].astype("category")
df_exog["weather"] = df_exog["weather"].astype("category")
df_exog["date"] = pd.to_datetime(df_exog["date"])
df_exog.set_index("date", inplace=True)
# set the datatime range index to monthly freq
df_exog.index = pd.date_range(start=df_exog.index.min(), end=df_exog.index.max(), freq='MS')
df_exog

,gw-level,month,year,quarter,season,weather,month_sin,month_cos,quarter_sin,quarter_cos,...,poly_quarter_month_sin,poly_quarter_month_cos,poly_quarter_quarter_sin,poly_quarter_quarter_cos,poly_month_sin_month_cos,poly_month_sin_quarter_sin,poly_month_sin_quarter_cos,poly_month_cos_quarter_sin,poly_month_cos_quarter_cos,poly_quarter_sin_quarter_cos
date,,,,,,,,,,,,,,,,,,,,,
1960-01-01,126.85,1,1960,1,winter,cold,2.588190e-01,9.659258e-01,1.000000e+00,6.123234e-17,...,2.588190e-01,9.659258e-01,1.000000e+00,6.123234e-17,2.500000e-01,2.588190e-01,1.584810e-17,9.659258e-01,5.914590e-17,6.123234e-17
1986-03-01,127.26,3,1986,1,spring,normal,7.071068e-01,7.071068e-01,1.000000e+00,6.123234e-17,...,7.071068e-01,7.071068e-01,1.000000e+00,6.123234e-17,5.000000e-01,7.071068e-01,4.329780e-17,7.071068e-01,4.329780e-17,6.123234e-17
1986-04-01,127.10,4,1986,2,spring,normal,8.660254e-01,5.000000e-01,1.224647e-16,-1.000000e+00,...,1.732051e+00,1.000000e+00,2.449294e-16,-2.000000e+00,4.330127e-01,1.060575e-16,-8.660254e-01,6.123234e-17,-5.000000e-01,-1.224647e-16
1986-05-01,126.96,5,1986,2,spring,normal,9.659258e-01,2.588190e-01,1.224647e-16,-1.000000e+00,...,1.931852e+00,5.176381e-01,2.449294e-16,-2.000000e+00,2.500000e-01,1.182918e-16,-9.659258e-01,3.169619e-17,-2.588190e-01,-1.224647e-16
1986-06-01,126.87,6,1986,2,summer,warm,1.000000e+00,6.123234e-17,1.224647e-16,-1.000000e+00,...,2.000000e+00,1.224647e-16,2.449294e-16,-2.000000e+00,6.123234e-17,1.224647e-16,-1.000000e+00,7.498799e-33,-6.123234e-17,-1.224647e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,NaN,8,2023,3,summer,warm,8.660254e-01,-5.000000e-01,-1.000000e+00,-1.836970e-16,...,2.598076e+00,-1.500000e+00,-3.000000e+00,-5.510911e-16,-4.330127e-01,-8.660254e-01,-1.590863e-16,5.000000e-01,9.184851e-17,1.836970e-16
2023-09-01,NaN,9,2023,3,Fall,normal,7.071068e-01,-7.071068e-01,-1.000000e+00,-1.836970e-16,...,2.121320e+00,-2.121320e+00,-3.000000e+00,-5.510911e-16,-5.000000e-01,-7.071068e-01,-1.298934e-16,7.071068e-01,1.298934e-16,1.836970e-16
2023-10-01,NaN,10,2023,4,Fall,normal,5.000000e-01,-8.660254e-01,-2.449294e-16,1.000000e+00,...,2.000000e+00,-3.464102e+00,-9.797174e-16,4.000000e+00,-4.330127e-01,-1.224647e-16,5.000000e-01,2.121150e-16,-8.660254e-01,-2.449294e-16


In [ ]:
# one-hot encoding for categorical features for transformer
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        make_column_selector(dtype_include=['category']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

# Lags grid
lags_grid = [24, 48, [1, 2, 3, 23, 24, 25, 125, 127, 128]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    }
    return search_space

In [ ]:
def search_hyperparameters(data, one_hot_encoder):

    # get the estimate end train and end validation dates
    exog_data = data.drop("gw-level", axis=1)
    exog_features = exog_data.columns
    df_idx = data.index
    train_num = int(len(data) * 0.8)
    end_train = "2012-12-01"
    end_valid = "2021-12-01"

    # instantiate a forcaster transformer with categorical features
    forecaster = ForecasterAutoreg(
        regressor = LGBMRegressor(random_state=15926, verbose=-1),
        lags = 72,
        transformer_exog = one_hot_encoder,
        fit_kwargs = {"categorical_feature": "auto"}
    )

    # search for best parameters
    results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_valid, 'gw-level'],
    exog               = data.loc[:end_valid, exog_features],
    search_space       = search_space,
    steps              = 24,
    refit              = False,
    metric             = 'mean_absolute_percentage_error',
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
    )
    best_params = results_search['params'].iat[0]

    return best_params, forecaster

best_params, forecaster =  search_hyperparameters(df_exog, one_hot_encoder)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'n_estimators': 1100, 'max_depth': 5, 'min_data_in_leaf': 254, 'learning_rate': 0.49292429494924545, 'feature_fraction': 0.8, 'max_bin': 175, 'reg_alpha': 0.1, 'reg_lambda': 0.9}
  Backtesting metric: 0.00102434567596089



/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [1, 2, 3, 23, 24, 25, 125, 127, 128] which is of type list.
  warnings.warn(message)


In [ ]:
data = df_exog.copy()
exog_data = data.drop("gw-level", axis=1)
exog_features = exog_data.columns
df_idx = data.index
train_num = int(len(data) * 0.8)
valid_num = len(data.loc[:"2021-11-01"])
end_train = df_idx[train_num]
end_valid = df_idx[valid_num]

In [ ]:
# train model with best params
forecaster = ForecasterAutoreg(
    regressor          = LGBMRegressor(**best_params, random_state=15926, verbose=-1),
    lags               = 72,
    transformer_exog   = one_hot_encoder,
    fit_kwargs         = {"categorical_feature": "auto"}
)
forecaster.fit(
    y    = data.loc[:end_valid, 'gw-level'],
    exog = data.loc[:end_valid, exog_features]
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# make predictions into the future
predictions = forecaster.predict(
    exog     = data.loc[df_idx[valid_num+1]:, exog_features],
    steps    = 24
)
pd.DataFrame(predictions)

,pred
2022-01-01,260.203161
2022-02-01,260.347878
2022-03-01,260.364267
2022-04-01,260.348622
2022-05-01,260.375187
2022-06-01,260.378496
2022-07-01,260.315726
2022-08-01,260.353924
2022-09-01,260.271444
2022-10-01,260.182519
